# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [73]:
# Libraries
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [74]:
# your code here
df = pd.read_csv('data\Admission_predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [75]:
# your code here
df.rename(columns=lambda x: x.strip(),inplace=True)

Let's evaluate the dataset by looking at the `head` function.

In [76]:
# your code here
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [77]:
# your code here
null_cols = df.isnull().sum()
null_cols

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [78]:
# your code here
df.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [79]:
# your code here
print(len(df['GRE Score']) == len(df['GRE Score'].drop_duplicates()))
print(len(df['CGPA']) == len(df['CGPA'].drop_duplicates()))
# so... individually they don't, maybe you mean together ?
print(sum(pd.Series(df['GRE Score'].astype(str) + df['CGPA'].astype(str)) != pd.Series(df['GRE Score'].astype(str) + df['CGPA'].astype(str)).drop_duplicates())==0)
# it seems you do.

False
False
True


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [80]:
# your code here
df.set_index(['GRE Score','CGPA'],inplace=True)
df.reset_index(inplace=True)
df.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [81]:
# your code here
sum((df['CGPA'] > 9) & (df['Research'] == 1))

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [82]:
# your code here
df[(df['CGPA'] > 9) & (df['SOP'] < 3.5)]

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
28,338,9.40,29,118,4,3.0,4.5,1,0.91
62,327,9.02,63,114,3,3.0,3.0,0,0.61
140,326,9.11,141,114,3,3.0,3.0,1,0.83
217,324,9.01,218,111,4,3.0,3.0,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84


In [83]:
df[(df['CGPA'] > 9) & (df['SOP'] < 3.5)]['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [84]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return (col - col.mean())/col.std()
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [85]:
# your code here
df['CGPA_std'] = standardize(df['CGPA'])
df['GRE_std'] = standardize(df['GRE Score'])
df['LOR_std'] = standardize(df['LOR'])

We will generate the decision choice at random using the code below. Please run the cell.

In [86]:
# Libraries
from random import choices

In [87]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']
decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [88]:
# your code here
df['deciding_column'] = df.lookup(df.index,decision_choice)

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [89]:
# your code here
df.loc[df['deciding_column'] > 0.8  , 'decision'] = 1
df.loc[df['deciding_column'] <= 0.8 , 'decision'] = 0
df.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.753381,1.0
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,-0.991211,0.0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,0.121034,0.0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597,0.0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,-0.480713,0.0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [90]:
# your code here
df['decision'].sum()

75.0

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [92]:
# your code here
df.columns=df.columns.str.replace(' ','_').str.replace('[^a-zA-Z_]','').str.lower()
df.columns
df.head()

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.753381,1.0
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,-0.991211,0.0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,0.121034,0.0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597,0.0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,-0.480713,0.0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [93]:
# your code here
df.loc[df['university_rating'] >= 4  , 'gre_score'] = df['gre_score'] + 10
df.head()

In [95]:
adjusted_gre=pd.cut(df['gre_score'],4)

In [97]:
adjusted_gre

0      (335.0, 350.0]
1      (305.0, 320.0]
2      (320.0, 335.0]
3      (305.0, 320.0]
4      (335.0, 350.0]
            ...      
380    (320.0, 335.0]
381    (320.0, 335.0]
382    (335.0, 350.0]
383    (305.0, 320.0]
384    (335.0, 350.0]
Name: gre_score, Length: 385, dtype: category
Categories (4, interval[float64]): [(289.94, 305.0] < (305.0, 320.0] < (320.0, 335.0] < (335.0, 350.0]]